# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [1]:
from ultralytics import YOLO, solutions
import cv2

In [5]:
!pip install ultralytics

In [7]:
# prompt: library opencv

!pip install opencv-python
!pip install gdown

In [8]:
!pip install ultralytics
!pip install opencv-p

ERROR: Could not find a version that satisfies the requirement opencv-p (from versions: none)
ERROR: No matching distribution found for opencv-p


## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [2]:
import cv2
from ultralytics import YOLO
model= YOLO('/content/YOLO_Model.pt')
# model = YOLO('yolov8n.pt')

In [3]:
model.info()

Model summary: 365 layers, 68,156,460 parameters, 0 gradients, 258.1 GFLOPs


(365, 68156460, 0, 258.13867519999997)

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [4]:
video_path = '/content/YOLO_Video.mp4'
cap = cv2.VideoCapture(video_path)

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [5]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

In [6]:
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [7]:
fps = int(cap.get(cv2.CAP_PROP_FPS))

## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [8]:
output_path = '/content/output.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

In [9]:
# video_writer = cv2.VideoWriter('outputCount.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [10]:
# w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
# up = h // 4
# line_points = [(20, up), (w - 20, up)]

In [11]:
# counter = solutions.ObjectCounter(
#     view_img=True,
#     reg_pts=line_points,
#     names=model.names,
#     draw_tracks=True,
#     line_thickness=2,
# )

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [12]:
import cv2
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_video.avi', fourcc, 20.0, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
color = (0, 255, 0)
thickness = 2
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.5
font_thickness = 1
ret = True

In [13]:
ret, frame = cap.read()
line_thickness = 2
top_line = 100
bottom_line = frame.shape[0] - 100  left_line = 100
right_line = frame.shape[1] - 100
top_counter = 0
bottom_counter = 0
left_counter = 0
right_counter = 0

In [15]:
while ret:
    ret, frame = cap.read()
    class_counts = {}
    results = model.track(frame, persist=True)
    for result in results:
        for obj in result.boxes:
            bbox = obj.xyxy[0].cpu().numpy()
            class_id = int(obj.cls[0].cpu().numpy()) if obj.cls is not None else -1
            conf = obj.conf[0].cpu().numpy() if obj.conf is not None else 0.0
            id = int(obj.id[0].cpu().numpy()) if obj.id is not None else -1
            if class_id not in class_counts:
                class_counts[class_id] = 0
            class_counts[class_id] += 1
            x1, y1, x2, y2 = map(int, bbox)
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            if center_y < top_line:
                top_counter += 1
            elif center_y > bottom_line:
                bottom_counter += 1
            elif center_x < left_line:
                left_counter += 1
            elif center_x > right_line:
                right_counter += 1
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)
            label = f'ID: {id} Class: {class_id} Conf: {conf:.2f}'
            cv2.putText(frame, label, (x1, y1 - 10), font, font_scale, color, font_thickness, cv2.LINE_AA)
    cv2.line(frame, (0, top_line), (frame.shape[1], top_line), color, line_thickness)
    cv2.line(frame, (0, bottom_line), (frame.shape[1], bottom_line), color, line_thickness)
    cv2.line(frame, (left_line, 0), (left_line, frame.shape[0]), color, line_thickness)
    cv2.line(frame, (right_line, 0), (right_line, frame.shape[0]), color, line_thickness)
    cv2.putText(frame, f' TOP: {top_counter}', (10, 30), font, font_scale, color, font_thickness, cv2.LINE_AA)
    cv2.putText(frame, f'DOWN : {bottom_counter}', (10, 50), font, font_scale, color, font_thickness, cv2.LINE_AA)
    cv2.putText(frame, f'LEFTT: {left_counter}', (10, 70), font, font_scale, color, font_thickness, cv2.LINE_AA)
    cv2.putText(frame, f'RIGHT: {right_counter}', (10, 90), font, font_scale, color, font_thickness, cv2.LINE_AA)
    out.write(frame)


0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 68.3ms
Speed: 3.9ms preprocess, 68.3ms inference, 600.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 55.6ms
Speed: 3.0ms preprocess, 55.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 55.6ms
Speed: 4.1ms preprocess, 55.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 50.1ms
Speed: 3.0ms preprocess, 50.1ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 50.0ms
Speed: 4.6ms preprocess, 50.0ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 82 cars, 2 trucks, 2 vans, 50.0ms
Speed: 3.0ms preprocess, 50.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 84 cars, 2 trucks, 2 vans, 42.0ms
Speed: 4.2ms preprocess, 42.0ms infere

In [ ]:

cap.release()
out.release()
cv2.destroyAllWindows()

## Save and Submit Your Work

Submit both the notebook and the output video

counting